# PYTHON LIBRARY IMPORT

In [1]:
import rasterio as rio
import numpy as np
import pandas as pd
import pickle
from collections import Counter
import os
import re

# Perform with All files

In [ ]:
# Define the preprocessing function
def preprocess(df, scaler_path):
    new_column_names = [
        'Band_3_Post', 'Band_4_Post', 'Band_5_Post', 'Band_6_Post',
        'Band_7_Post', 'Band_8_Post', 'Band_8A_Post', 'Band_9_Post', 'Band_12_Post'
    ]
    rename_dict = dict(zip(df.columns[:len(new_column_names)], new_column_names))
    df_rename = df.rename(columns=rename_dict)

    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)

    df_normalized = scaler.transform(df_rename)
    df_normalized = pd.DataFrame(df_normalized, columns=df_rename.columns)
    df_rename = df_normalized.rename(columns=rename_dict)

    return df_rename

# Define the prediction function
def make_predictions(directory, df_rename):
    loaded_model = pickle.load(open(directory, 'rb'))
    y_pred = loaded_model.predict(df_rename)
    df_rename["BURN_PREDICTED"] = y_pred
    label_counts = Counter(y_pred)
    print("\nCount of each predicted label:")
    for label, count in label_counts.items():
        print(f"Label {label}: {count}")

    return df_rename

# Define the function to create GeoTIFF from predictions
def create_geotiff_from_predictions(predictions, original_tif_path, output_tif_path):
    with rio.open(original_tif_path) as src:
        original_metadata = src.meta.copy()
    
    original_metadata.update({
        'dtype': 'uint8',
        'count': 1,
    })

    band_1 = predictions.values.reshape((original_metadata['height'], original_metadata['width']))

    with rio.open(output_tif_path, 'w', **original_metadata) as new_img:
        new_img.write(band_1, 1)

    print(f"New GeoTIFF file '{output_tif_path}' has been created.")

# Directory containing the raster files
raster_dir = r"raster"
scaler_path = r"model/min_max_scaler.pkl"
model_path = r"model/Model_LGBM.sav"

# Loop through each file in the directory
for file_name in os.listdir(raster_dir):
    if file_name.endswith(".tif"):
        file_path = os.path.join(raster_dir, file_name)

        # Extract year from the filename using regex
        match = re.search(r'_(\d{4})', file_name)
        if match:
            year = match.group(1)

            # Process the file
            img = rio.open(file_path)
            array = img.read()
            n_bands = array.shape[0]

            df = pd.DataFrame(array.reshape([n_bands, -1]).T, columns=[f"band_{i+1}" for i in range(n_bands)])
            df_rename = preprocess(df, scaler_path)
            df_predicted = make_predictions(model_path, df_rename)

            # Create GeoTIFF file from predictions
            output_tif_path = os.path.join(raster_dir, f'output_{year}.tif')
            create_geotiff_from_predictions(df_predicted["BURN_PREDICTED"], file_path, output_tif_path)